In [19]:
from hpsklearn import HyperoptEstimator, extra_trees, any_classifier, pca, random_forest
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, make_scorer
from hyperopt import tpe
import numpy as np
import pandas as pd


In [20]:
import sklearn

In [21]:
sklearn.__version__

'0.20.3'

In [22]:
def loss_function(targ, pred):
    # hyperopt_estimator flattens the prediction when saving it.  This also
    # affects multilabel classification.
    pred = pred.reshape( (-1, 2) )
    return log_loss(targ, pred[:,1])

In [23]:
def rocauc_loss(targ, pred):
    pred = pred.reshape( (-1, 2) )
    return 1 - roc_auc_score(targ, pred[:,1])

In [24]:

# Download the data and split into training and test sets
df = pd.read_csv('./genetic_algorithm/train.csv')


target = 'Survived'
predictors = list(set(df._get_numeric_data().columns) - set([target,'PassengerId']))

X_train, X_test, y_train, y_test = train_test_split(df[predictors].fillna(0),df[target],test_size=0.1)


In [27]:
y_train

459    0
842    1
271    1
755    1
498    0
412    1
371    0
128    1
78     1
312    0
149    0
834    0
24     0
746    0
546    1
627    1
780    1
806    0
583    0
437    1
460    1
825    0
813    0
429    1
628    0
211    1
824    0
817    0
835    1
539    1
      ..
591    1
481    0
512    1
808    0
264    0
142    1
580    1
16     0
558    1
197    0
324    0
159    0
67     0
471    0
637    0
419    0
573    1
35     0
832    0
615    1
509    1
279    1
240    0
727    1
884    0
538    0
153    0
659    0
345    1
12     0
Name: Survived, Length: 801, dtype: int64

In [28]:

# Instantiate a HyperoptEstimator with the search space and number of evaluations

estim = HyperoptEstimator(classifier=random_forest('clf'),
                          preprocessing=[],
                          max_evals=10,
                          trial_timeout=100,
                          continuous_loss_fn = True,
                          loss_fn=rocauc_loss
                         )

# Search the hyperparameter space based on the data
estim.fit( X_train.values, y_train.values )

# Show the results

print(estim.score(X_test, y_test))
# 0.962785714286 

print(estim.best_model())
# {'learner': ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='entropy',
#           max_depth=None, max_features=0.959202875857,
#           max_leaf_nodes=None, min_impurity_decrease=0.0,
#           min_impurity_split=None, min_samples_leaf=1,
#           min_samples_split=2, min_weight_fraction_leaf=0.0,
#           n_estimators=20, n_jobs=1, oob_score=False, random_state=3,
#           verbose=False, warm_start=False), 'preprocs': (), 'ex_preprocs': ()}

  0%|                                                                              | 0/1 [00:00<?, ?it/s, best loss: ?]


IOError: [Errno 22] Invalid argument

In [8]:
estim.best_model()['learner']


RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='entropy', max_depth=None,
            max_features=0.743852748379, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2183, n_jobs=-1,
            oob_score=False, random_state=1, verbose=False,
            warm_start=False)

In [9]:
%matplotlib inline

In [ ]:
pd.DataFrame(zip(X_train.columns,estim.best_model()['learner'].feature_importances_)).rename(columns={0:'feature',1:'importance'}).set_index('feature').sort_values('importance').plot.barh()

In [11]:
1-rocauc_loss(y_test, estim.best_model()['learner'].predict_proba(X_test))

0.7076671035386632